# Solving the Dice Game Pig with Dynamic Programming
*This notebook is going to be updated frequently, stay tuned*.

v2 Changes:
- get_observations now returns observations as a tuple so it can be used as a key for a dictionary
- Implemented a simple 'Hold at 20' agent that can interact with the environment.
- Computed the state-value function for the hold at 20 strategy vs the same hold at 20 strategy.

In [14]:
import random

class Pig:
    def __init__(self):
        self.state = [0, 0, 0]
        self.current_player = 1
        self.player_names = ("Player 1", "Player 2")
        self.history = []
        
    def roll(self):
        dice_value = random.randint(1, 6)
        if dice_value == 1:
            self.state[2] = 0
        else:
            self.state[2] += dice_value
        return dice_value
        
    def hold(self):
        self.state[0] += self.state[2]
        self.state[2] = 0
        return 0  # 0 indicates hold
    
    def change_player(self):
        self.state[0], self.state[1] = self.state[1], self.state[0]
        self.current_player = 3 - self.current_player
    
    def action(self, action):
        assert action in [0, 1], "Invalid move selected."  
        assert not self.is_done(), "Game is over. Actions not possible."
            
        if action == 0:
            roll_value = self.hold()
        elif action == 1:
            roll_value = self.roll()
            
        info = self.get_info(action, roll_value)
        
        if roll_value in [0, 1]:  # if hold or roll a 1
            if self.state[0] < 100:
                self.change_player()
        
        observations = self.get_observations()
        reward = self.calculate_reward()
        done = self.is_done()
        
        return observations, reward, done, info
        
    def get_observations(self):
        return tuple(self.state.copy())
    
    def get_actions(self):
        return [] if self.is_done() else [0, 1]
            
    def reset(self):
        self.state = [0, 0, 0]
        self.current_player = 1
        return self.get_observations()
    
    def calculate_reward(self):
        if self.state[0] >= 100:
            return 1
        elif self.state[1] >= 100:
            return -1
        else:
            return 0
        
    def get_info(self, action, roll_value):
        info = self.player_names[self.current_player-1]
        if action == 0:
            info += f" holds, bringing their total score to {self.state[0]}."
        elif action == 1 and roll_value != 1:
            info += f" rolls a {roll_value}, bringing their turn score to {self.state[2]} and total score to {self.state[0] + self.state[2]}."
        elif action == 1 and roll_value == 1:
            info += f" rolls a 1, losing their turn score, resetting their total score to {self.state[0] + self.state[2]}."
        return info
        
    def is_done(self):
        return self.state[0] >= 100 or self.state[1] >= 100
    
    
from collections import defaultdict

class HoldOn20Agent:
    def __init__(self):
        self.reward = 0
        self.policy = defaultdict(int)  # Initialize policy dictionary
        
        # Populate policy dictionary with all possible states for plotting
        for p_score in range(0, 106):
            for o_score in range(0, 106):
                for t_total in range(0, 106):
                    if p_score + t_total  >= 100:
                        self.policy[(p_score, o_score, t_total)] = 0 # Hold and win
                    if t_total >= 20:
                        self.policy[(p_score, o_score, t_total)] = 0 # Hold
                    else:
                        self.policy[(p_score, o_score, t_total)] = 1 # Roll
                        
    def reset(self):
        self.reward = 0

    def step(self, env: Pig):
        # Get observations from environment to make decision with (discarded with random agent)
        state = env.get_observations()

        # Get available actions and choose a random one
        actions = env.get_actions()

        action = self.policy[state]

        obs, reward, done, info = env.action(action)
        self.reward += reward

## Modelling this as a Reinforcement Learning Problem

The game of Pig lends itself naturally to the framework of reinforcement learning. In reinforcement learning, an agent (the player in our case) interacts with an environment (the game of Pig), receiving feedback in terms of rewards (winning or losing the game) and tries to learn an optimal strategy over time.

In our context, we can formulate our problem in the reinforcement learning context as follows:

- **State (S)**: A state is defined as a tuple of three elements (i, j, k) where `i` is the current player's score, `j` is the opponent's score, and `k` is the total score accumulated so far in the current turn.

- **Action (A)**: There are two possible actions for the player at any state: 'roll' or 'hold'. 

- **Reward (R)**: The reward function is binary. A reward of `1` is given when the player wins (i.e., when the player's score is equal to or more than 100), and `0` otherwise.

- **Policy (π)**: The policy defines the player's behavior at a given state. In this case, the policy is "hold on 20", which means the player will hold if the turn total `k` is more than or equal to 20, or if the player's score `i` is greater than or equal to 100.

- **State-Value Function (V)**: The state-value function `V(s)` under a policy `π` is the expected return when starting in state `s` and following `π` thereafter. For us, this will indicate the probability of winning from a given state following our policy.

---

## Modelling the Problem as a Markov Decision Process (MDP)

A Markov Decision Process (MDP) provides a mathematical framework for modeling decision-making in situations where outcomes are partly random and partly under the control of a decision-maker. It is particularly well suited for this game. We can define our MDP as follows:

- **States (S)** and **Actions (A)**: Defined as above.

- **Transition Probability (P)**: If we denote `s` and `s'` as the initial and subsequent states respectively, and `a` as the action taken, then our transition probability `P(s'|s,a)` is as follows:
  - If `a = roll`, and the dice result is `d`, then `s' = (i, j, k + d)` with probability `1/6`.
  - If `a = roll`, and the dice result is `1`, then `s' = (j, i, 0)` with probability `1/6`.
  - If `a = hold`, then `s' = (j, i + k, 0)` with probability `1`.

- **Reward Function (R)**: As defined above.

The core of our approach will involve **iterative policy evaluation**, which is a standard method in dynamic programming for estimating the state-value function `V(s)` under a given policy. We will use this method to analyze the "hold on 20" strategy. 

---

## Implementing Iterative Policy Evaluation

Given our policy (hold on 20), we want to compute the state-value function `V(s)`, which represents the probability of winning from state `s` under our policy. We initialize all state values to 0, except for terminal states which are set to 1 for winning states and 0 for losing states.

We then use the Bellman expectation equation for iterative policy evaluation:

$$
V_{k+1}(s) = \sum_{a \in A} \pi(a|s) \sum_{s' \in S} P(s'|s,a) [R(s,a,s') + \gamma V_k(s')]
$$

Since our rewards are only at the end of the game, we can simplify the equation to:

$$
V_{k+1}(s) = \sum_{a \in A} \pi(a|s) \sum_{s' \in S} P(s'|s,a) V_k(s')
$$

Here, `V_k(s)` is the value of state `s` at the `k`-th iteration of the algorithm. `P(s'|s,a)` is the probability of transitioning to state `s'` when action `a` is taken at state `s`. `π(a|s)` represents the probability of taking action `a` at state `s` under policy `π`, which in our case is deterministic.

This process is repeated until the maximum change in state value function is less than a small threshold, at which point the value function has converged to the true state-value function under the given policy.

To summarize in 4 steps:

1. **Initialization**: All state values, except for terminal states, are initialized to 0. Terminal states are initialized to 1 if they represent a win and 0 if they represent a loss.
2. **Policy Evaluation Step**: For each state, we calculate the expected value of each possible action under the current policy. This involves calculating the transition probabilities for each action, and using the current estimates of the state-value function.
3. **Update Step**: We use the computed values to update our estimates of the state-value function. The largest change in the state-value function across all states is recorded.
4. **Convergence Check**: The process of steps 2 and 3 is repeated until the largest change in the state-value function is less than a small threshold (delta), indicating that our estimates have converged.

Once we have implemented the above steps, we can estimate the value of any state for two players following the "hold on 20" policy. For example, to estimate the value of the state (50, 50, 10), we perform the following operations:

In [80]:
import numpy as np

# Probability of winning in state (i, j, k)
# We include up to 105 into each state as it's technically possible to reach 105 with this strategy
p = np.zeros((105, 105, 105))

# Example state
state = (50, 50, 10) 

i, j, k = state

# Terminal states
if i + k >= 100:
    p[i, j, k] = 1.0
elif j >= 100:
    p[i, j, k] = 0.0
    
# Else follow the policy
else:
    
    # Hold if our current turn total is over 20
    if k >= 20:
        
        # The probability the player wins is equal to the probability the opponent doesn't win
        p[i, j, k] = 1 - p[j, i + k, 0]
        
    # Else we roll
    else:
        
        # Rolling a 1
        p_roll = (1 - p[j, i, 0]) / 6
        
        # Rolling a 2
        p_roll += p[i, j, k + 2] / 6
        
        # Rolling a 3
        p_roll += p[i, j, k + 3] / 6
        
        # Rolling a 4
        p_roll += p[i, j, k + 4] / 6
        
        # Rolling a 5
        p_roll += p[i, j, k + 5] / 6
        
        # Rolling a 6
        p_roll += p[i, j, k + 6] / 6
        
        # Update the current estimate from these values
        p[i, j, k] = p_roll
        
print("Estimate of winning from a single iteration:", p[i,j,k])

Estimate of winning from a single iteration: 0.16666666666666666


This process can be repeated for all states to generate the complete state-value function for the "hold on 20" policy. Once we have this, we can analyze the effectiveness of the policy and potentially explore improvements or alternatives.


#### Improving convergence speed through backward induction
Given that our iterative policy evaluation technique relies on bootstrapping from existing estimates, and that initially only the terminal states possess non-zero estimates, we can significantly accelerate the convergence of our method by iterating through the game states in reverse order.

This method, known as Backward Induction, starts from the maximum possible state, (105, 105, 105) in our case, and moves backwards to the initial state, (0, 0, 0). The primary advantage of this technique is that it allows for the immediate propagation of the value estimates from the terminal states to their predecessors, thereby eliminating the need for the value estimates to slowly propagate forward from the terminal states.

In [111]:
import numpy as np

# Initialize the state-value function array to zeros
V = np.zeros((100 + 6, 100 + 6, 100 + 6))

# Small number for stopping condition
theta = 1e-5  
delta = 1 

# We store all computed max deltas for plotting later
delta_list = []

while delta > theta:
    delta = 0

    for i in reversed(range(100 + 6)): # Backward induction
        for j in reversed(range(100 + 6)):
            for k in reversed(range(100 + 6)):

                # Copy old value to calculate change later
                v_old = V[i, j, k]

                # If player's score is >= 100, they've won
                if i >= 100:
                    V[i, j, k] = 1.0
                    continue

                # If opponent's score is >= 100, then player has lost
                if j >= 100:
                    V[i, j, k] = 0.0
                    continue

                # If the turn score is >= 20 or if holding would result in winning the game
                if k >= 20 or i + k >= 100:
                    # The new state is (j, i + k, 0) (opponent's turn)
                    V[i, j, k] = 1 - V[j, min(100, i + k), 0]
                else:
                    # The player will roll the dice according to the policy
                    # Consider all possible outcomes: rolling 1, 2, 3, 4, 5, or 6
                    v_roll = (1 - V[j, i, 0]) / 6  # Rolling a 1
                    for roll in range(2, 7):  # Rolling 2, 3, 4, 5, 6
                        v_roll += V[i, j, k + roll] / 6

                    V[i, j, k] = v_roll

                # Update delta
                delta = max(delta, abs(v_old - V[i, j, k]))
                
    delta_list.append(delta)

    
import plotly.graph_objects as go

def plot_policy_evaluation_convergence(delta_values, title = ""):
    # Create a line plot
    fig = go.Figure()

    # Add trace
    fig.add_trace(go.Scatter(
        y=delta_values,  # errors on y-axis
        mode='lines+markers',  # line plot with markers
        marker=dict(
            size=8,  # size of the markers
            color='#00CC96',  # color of the markers
        ),
        line=dict(
            width = 3,
            color='#00CC96',  # color of the line
        ),
        hovertemplate='Iterations: %{x}<br>Largest Prob Update: %{y}',  # custom hover template
        name='Largest Probability Update',  # name of the trace
    ))

    # Set title and labels
    fig.update_layout(
        title='Policy Evaluation ' + title,
        xaxis_title='Iterations',
        yaxis_title='Largest Probability Update',
    )

    # Set y-axis to logarithmic scale
    fig.update_yaxes(type='log')

    # Show the plot
    fig.show()
    
plot_policy_evaluation_convergence(delta_list, "- Hold on 20")